In [94]:
import sys
import os
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")
sys.path.append(current_dir)
from Preprocessing_pipeline import PreprocessingPipeline

Current directory: d:\Dicoding\AURA\Src


Load model

In [95]:
def load_models():
    models = {}
    try:
        # Load preprocessing pipeline
        print("\n[1/5] Loading preprocessing pipeline...")
        models['pipeline'] = PreprocessingPipeline.load('../Pipeline/preprocessing_pipeline.pkl')
        print("Pipeline loaded")
        
        # Load scaler
        print("\n[2/5] Loading scaler...")
        models['scaler'] = joblib.load('../Model/scaler.pkl')
        print("Scaler loaded")
        
        # Load RUL model
        print("\n[3/5] Loading RUL model...")
        models['rul_model'] = joblib.load('../Model/rul_model.pkl')
        print("RUL model loaded")
        
        # Load failure classifier
        print("\n[4/5] Loading failure classifier...")
        models['failure_model'] = joblib.load('../Model/failure_model.pkl')
        print("Failure classifier loaded")
        
        # Load label encoder
        print("\n[5/5] Loading label encoder...")
        models['label_encoder'] = joblib.load('../Model/label_encoder.pkl')
        print("Label encoder loaded")
        
        return models
        
    except Exception as e:
        print(f"\n ERROR loading models: {str(e)}")
        sys.exit(1)

In [96]:
def create_test_data():
    """Create dummy test data with various scenarios"""
    
    test1 = {
        'name': 'Scenario 1',
        'description': 'New machine, optimal conditions',
        'data': {
            'datetime': '2025-01-20 10:00:00',
            'Type': 'M',
            'Air_temperature': 300.0,
            'Process_temperature': 310.0,
            'Rotational_speed': 1500,
            'Torque': 40.0,
            'Tool_wear': 50,
            'machine_age_hours': 1000,
            'hours_since_last': 8,
            'Temp_Rate_of_Change': 0.0,
            'RPM_Variance': 15.0
        }
    }

    test2 = {
        'name': 'Scenario 2',
        'description': 'Aging machine, moderate wear',
        'data': {
            'datetime': '2025-01-20 14:30:00',
            'Type': 'H',
            'Air_temperature': 302.0,
            'Process_temperature': 310.0,
            'Rotational_speed': 1450,
            'Torque': 45.0,
            'Tool_wear': 180,
            'machine_age_hours': 15000,
            'hours_since_last': 8,
            'Temp_Rate_of_Change': 0.5,
            'RPM_Variance': 35.0
        },
    }
    
    # Scenario 3: Critical condition (short RUL)
    test3 = {
        'name': 'Scenario 3',
        'description': 'High wear, temperature issues',
        'data': {
            'datetime': '2025-01-20 18:45:00',
            'Type': 'L',
            'Air_temperature': 310.0,
            'Process_temperature': 322.0,
            'Rotational_speed': 1400,
            'Torque': 52.0,
            'Tool_wear': 280,
            'machine_age_hours': 22000,
            'hours_since_last': 8,
            'Temp_Rate_of_Change': 1.2,
            'RPM_Variance': 55.0
        },
    }
    
    test4 = {
        'name': 'Scenario 4',
        'description': 'High RPM, moderate load',
        'data': {
            'datetime': '2025-01-20 22:15:00',
            'Type': 'M',
            'Air_temperature': 298.0,
            'Process_temperature': 308.0,
            'Rotational_speed': 2800,
            'Torque': 35.0,
            'Tool_wear': 120,
            'machine_age_hours': 8000,
            'hours_since_last': 8,
            'Temp_Rate_of_Change': 0.2,
            'RPM_Variance': 45.0
        },
    }
    
    test5 = {
        'name': 'Scenario 5',
        'description': 'Heavy load, high torque',
        'data': {
            'datetime': '2025-01-21 08:00:00',
            'Type': 'H',
            'Air_temperature': 302.0,
            'Process_temperature': 314.0,
            'Rotational_speed': 1350,
            'Torque': 68.0,
            'Tool_wear': 200,
            'machine_age_hours': 18000,
            'hours_since_last': 8,
            'Temp_Rate_of_Change': 0.8,
            'RPM_Variance': 40.0
        },
    }
    
    test_cases = [test1, test2, test3, test4, test5]
    
    print(f"\nCreated {len(test_cases)} test scenarios")
    
    return test_cases


In [97]:
def predict_maintenance(models, sensor_data):
    """
    Complete prediction pipeline
    
    Args:
        models: Dictionary of loaded models
        sensor_data: Dictionary with sensor readings
        
    Returns:
        Dictionary with prediction results
    """
    
    try:
        # 1. Validate input
        is_valid, error_msg = models['pipeline'].validate_input(sensor_data)
        if not is_valid:
            return {
                "status": "error",
                "error": error_msg
            }
        
        # 2. Transform to features
        features = models['pipeline'].transform_single(sensor_data)
        
        # 3. Scale features
        features_scaled = models['scaler'].transform(features)
        
        # 4. Predict RUL
        rul_hours = float(models['rul_model'].predict(features_scaled)[0])
        rul_days = rul_hours / 24
        
        # 5. Predict failure type (if RUL < 2 months)
        failure_type = None
        failure_confidence = None
        failure_probabilities = None
        
        if rul_days < 60:  # Critical period
            failure_idx = models['failure_model'].predict(features_scaled)[0]
            failure_proba = models['failure_model'].predict_proba(features_scaled)[0]
            failure_type = models['label_encoder'].inverse_transform([failure_idx])[0]
            failure_confidence = float(failure_proba[failure_idx])
            
            # Get all probabilities
            failure_classes = models['label_encoder'].classes_
            failure_probabilities = {
                cls: float(prob) 
                for cls, prob in zip(failure_classes, failure_proba)
            }
        
        # 6. Determine status and action
        if rul_days < 7:
            status = "CRITICAL"
            priority = "URGENT"
            action = "Schedule maintenance IMMEDIATELY (within 1-2 days)"
        elif rul_days < 30:
            status = "CRITICAL"
            priority = "HIGH"
            action = "Schedule maintenance within 1-2 weeks"
        elif rul_days < 60:
            status = "WARNING"
            priority = "MEDIUM"
            action = "Schedule maintenance within 4-8 weeks"
        else:
            status = "NORMAL"
            priority = "LOW"
            action = "Continue routine monitoring"
        
        # 7. Build response
        return {
            "status": "success",
            "prediction": {
                "rul_hours": round(rul_hours, 1),
                "rul_days": round(rul_days, 1),
                "status": status,
                "priority": priority,
                "action": action
            },
            "failure": {
                "type": failure_type,
                "confidence": round(failure_confidence, 3) if failure_confidence else None,
                "probabilities": {
                    k: round(v, 3) 
                    for k, v in failure_probabilities.items()
                } if failure_probabilities else None
            },
            "sensor_summary": {
                "air_temp": sensor_data['Air_temperature'],
                "process_temp": sensor_data['Process_temperature'],
                "rpm": sensor_data['Rotational_speed'],
                "torque": sensor_data['Torque'],
                "tool_wear": sensor_data['Tool_wear'],
                "machine_type": sensor_data['Type']
            },
            "timestamp": sensor_data.get('datetime')
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error": f"Prediction failed: {str(e)}"
        }

In [98]:
def print_prediction_result(test_case, result, test_num, total_tests):
    """Print formatted prediction result"""
    
    print(f"TEST {test_num}/{total_tests}: {test_case['name']}")
    
    # Test info
    print(f"\nDescription: {test_case['description']}")
    
    # Input summary
    print(f"\nINPUT:")
    data = test_case['data']
    print(f"   Machine Type:     {data['Type']}")
    print(f"   Air Temp:         {data['Air_temperature']}K")
    print(f"   Process Temp:     {data['Process_temperature']}K")
    print(f"   RPM:              {data['Rotational_speed']}")
    print(f"   Torque:           {data['Torque']} Nm")
    print(f"   Tool Wear:        {data['Tool_wear']} min")
    print(f"   Machine Age:      {data['machine_age_hours']} hours")
    
    # Check if prediction succeeded
    if result['status'] != 'success':
        print(f"\nPREDICTION FAILED!")
        print(f"   Error: {result.get('error', 'Unknown error')}")
        return
    
    # Prediction results
    pred = result['prediction']
    print(f"\nPREDICTION:")
    print(f"   RUL (hours):      {pred['rul_hours']:.1f} hours")
    print(f"   RUL (days):       {pred['rul_days']:.1f} days")
    print(f"   Status:           {pred['status']}")
    print(f"   Priority:         {pred['priority']}")
    print(f"   Action:           {pred['action']}")
    
    # Failure prediction (if available)
    if result['failure']['type']:
        print(f"\nFAILURE PREDICTION:")
        print(f"   Type:             {result['failure']['type']}")
        print(f"   Confidence:       {result['failure']['confidence']:.1%}")
        
        if result['failure']['probabilities']:
            print(f"\n   All Probabilities:")
            for failure_type, prob in sorted(
                result['failure']['probabilities'].items(), 
                key=lambda x: x[1], 
                reverse=True
            ):
                bar_length = int(prob * 40)
                bar = "█" * bar_length + "░" * (40 - bar_length)
                print(f"   {failure_type:25s} {bar} {prob:.1%}")
    else:
        print(f"\n✓ No immediate failure detected (RUL > 60 days)")
    
    # Status indicator
    if pred['status'] == 'CRITICAL':
        print(f"\n🔴 CRITICAL: Immediate attention required!")
    elif pred['status'] == 'WARNING':
        print(f"\n🟡 WARNING: Plan maintenance soon")
    else:
        print(f"\n🟢 NORMAL: Machine operating normally")
    
    print()

In [99]:
def run_tests():
    """Main test execution"""
    
    # Load models
    models = load_models()
    
    # Create test data
    test_cases = create_test_data()
    
    # Run tests
    print("RUNNING TESTS")
    
    results = []
    for i, test_case in enumerate(test_cases, 1):
        result = predict_maintenance(models, test_case['data'])
        results.append({
            'test_case': test_case,
            'result': result
        })
        print_prediction_result(test_case, result, i, len(test_cases))
    
    # Summary
    print("TEST SUMMARY")
    
    successful = sum(1 for r in results if r['result']['status'] == 'success')
    failed = len(results) - successful
    
    print(f"\n RESULTS:")
    print(f"   Total tests:      {len(results)}")
    print(f"   Successful:       {successful} ✓")
    print(f"   Failed:           {failed} {'✗' if failed > 0 else ''}")
    print(f"   Success rate:     {successful/len(results)*100:.1f}%")
    
    # Status distribution
    print(f"\n STATUS DISTRIBUTION:")
    status_counts = {}
    for r in results:
        if r['result']['status'] == 'success':
            status = r['result']['prediction']['status']
            status_counts[status] = status_counts.get(status, 0) + 1
    
    for status, count in sorted(status_counts.items()):
        print(f"   {status:15s} {count:2d} {'🔴' if status=='CRITICAL' else '🟡' if status=='WARNING' else '🟢'}")
    
    # RUL range
    print(f"\n RUL RANGE:")
    ruls = [r['result']['prediction']['rul_days'] 
            for r in results if r['result']['status'] == 'success']
    if ruls:
        print(f"   Min RUL:          {min(ruls):.1f} days")
        print(f"   Max RUL:          {max(ruls):.1f} days")
        print(f"   Average RUL:      {np.mean(ruls):.1f} days")
    
    print("\n" + "="*80)
    if successful == len(results):
        print(" ALL TESTS PASSED SUCCESSFULLY!")
    else:
        print(f" {failed} TEST(S) FAILED - REVIEW ERRORS ABOVE")
    print("="*80)
    
    return results

In [100]:
if __name__ == "__main__":
    print("PREDICTIVE MAINTENANCE - MODEL TESTING")
    
    try:
        results = run_tests()
    except KeyboardInterrupt:
        print("\n\n Testing interrupted by user")
        sys.exit(1)
    except Exception as e:
        print(f"\n\n ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

PREDICTIVE MAINTENANCE - MODEL TESTING

[1/5] Loading preprocessing pipeline...
✓ Preprocessing pipeline loaded from: ../Pipeline/preprocessing_pipeline.pkl
Pipeline loaded

[2/5] Loading scaler...
Scaler loaded

[3/5] Loading RUL model...
RUL model loaded

[4/5] Loading failure classifier...
Failure classifier loaded

[5/5] Loading label encoder...
Label encoder loaded

Created 5 test scenarios
RUNNING TESTS
TEST 1/5: Scenario 1

Description: New machine, optimal conditions

INPUT:
   Machine Type:     M
   Air Temp:         300.0K
   Process Temp:     310.0K
   RPM:              1500
   Torque:           40.0 Nm
   Tool Wear:        50 min
   Machine Age:      1000 hours

PREDICTION:
   RUL (hours):      7452.9 hours
   RUL (days):       310.5 days
   Status:           NORMAL
   Priority:         LOW
   Action:           Continue routine monitoring

✓ No immediate failure detected (RUL > 60 days)

🟢 NORMAL: Machine operating normally

TEST 2/5: Scenario 2

Description: Aging machine,